In [117]:
import numpy as np

In [118]:
class ScratchSVMClassifier():
    """
    Scratch implementation of SVM classifier
    Parameters
    ----------
    num_iter : int
      Number of iterations
    lr : float
      Learning rate
    kernel : str
      Kernel type. Linear kernel (linear) or polynomial kernel (polly)
    threshold : float
      Threshold for choosing a support vector
    verbose : bool
      True to output the learning process
    Attributes
    ----------
    self.n_support_vectors : int
      Number of support vectors
    self.index_support_vectors : The following form of ndarray, shape (n_support_vectors,)
      Support vector index
    self.X_sv :  The following forms of ndarray, shape (n_support_vectors, n_features)
      Support vector features
    self.lam_sv :  The following forms of ndarray, shape (n_support_vectors, 1)
      Support vector undetermined multiplier
    self.y_sv :  The following forms of ndarray, shape (n_support_vectors, 1)
      Support vector label
    """
    def __init__(self, num_iter=None, lr=None, kernel='linear', threshold=1e-5, verbose=False, random_state=None):
        # Record hyperparameters as attributes
        self.iter = num_iter
        self.lr = lr
        self.kernel = kernel
        self.threshold = threshold
        self.verbose = verbose
        self.random_state = random_state

    def _init_lambda(self, higher, size):
        np.random.seed(self.random_state)
        return np.random.uniform(0, higher, size)

    def _linear_kernel(self, X_i, X_j):
        return X_i @ X_j.T
    
    def _poly_kernel(self, X_i, X_j):
        return

    def _kernel_chooser(self, X_i, X_j):
        if (self.kernel == 'linear'):
            return self._linear_kernel(X_i, X_j)
        elif (self.kernel == 'poly'):
            return self._poly_kernel(X_i, X_j)
        else:
            raise Exception('Invalid kernel')

    def _Lagrange_sum(self, X, y, _lambda):
        s = np.zeros(y.shape[0])
        for i in range(0, y.shape[0]):
            for j in range(0, y.shape[0]):
                s[i] = _lambda[j] * y[j] * self._kernel_chooser(X[i, :], X[j, :])
        return s

    def _diff(self, X, y, _lambda):
        return 1 - y * self._Lagrange_sum(X, y, _lambda)

    def _gradient_ascent(self, X, y):
        _lambda = self._init_lambda(10, y.shape[0])
        for i in range(0, self.iter):
            _lambda = _lambda + self.lr * self._diff(X, y, _lambda)
            _lambda[_lambda < 0] = 0
        return _lambda

    def _determine_support_vectors(self, X, y):
        _lambda = self._gradient_ascent(X, y)
        idx = np.where(_lambda > self.threshold)
        return X[idx], y[idx], _lambda[idx]

    def fit(self, X, y, X_val=None, y_val=None):
        """
        Learn the SVM classifier. If verification data is input, the accuracy for it is also calculated for each iteration.
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            Features of training data
        y : The following form of ndarray, shape (n_samples,)
            Correct answer value of training data
        X_val : 次の形のndarray, shape (n_samples, n_features)
            Features of verification data
        y_val : The following form of ndarray, shape (n_samples,)
            Correct value of verification data
        """
        self._y_unique = np.unique(y)
        y = np.where(y == self._y_unique[0], -1, 1)

        if self.verbose:
            #Output the learning process when verbose is set to True print()
            print("training set: fitting...")
            self.sup_vectors, self.sup_vectors_label, self.sup_lambda = self._determine_support_vectors(X, y)
            if ((X_val is not None) and (y_val is not None)):
                print("test set: fitting...")
                self.sup_vectors_val, self.sup_vectors_label_val, self.sup_lambda_val = self._determine_support_vectors(X_val, y_val)

        else:
            self.sup_vectors, self.sup_vectors_label, self.sup_lambda = self._determine_support_vectors(X, y)
            if ((X_val is not None) and (y_val is not None)):
                self.sup_vectors_val, self.sup_vectors_label_val, self.sup_lambda_val = self._determine_support_vectors(X_val, y_val)

    def predict(self, X):
        """
        Estimate the label using the SVM classifier.
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            sample
        Returns
        -------
            The following form of ndarray, shape (n_samples, 1)
            Estimated result by SVM classifier
        """
        result = np.zeros(X.shape[0])
        for n in range(0, self.sup_lambda.shape[0]):
            result += self.sup_lambda[n] * self.sup_vectors_label[n] * self._kernel_chooser(X, self.sup_vectors[n])
        
        result = np.where(result < 0, self._y_unique[0], self._y_unique[1])
        return result

<h3>【problem 4】Learning and estimation</h3>

In [119]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics

<h4>Dataset 1</h4>

In [120]:
np.random.seed(seed=0)
n_samples = 500
f0 = [-1, 2]
f1 = [2, -1]
cov = [[1.0,0.8], [0.8, 1.0]]
f0 = np.random.multivariate_normal(f0, cov, n_samples // 2)
f1 = np.random.multivariate_normal(f1, cov, n_samples // 2)
X = np.concatenate([f0, f1])
y = np.concatenate([
    np.full(n_samples // 2, 1),
    np.full(n_samples // 2, -1)
])

In [121]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=69)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

scaler = StandardScaler()
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

(375, 2)
(125, 2)
(375,)
(125,)


In [122]:
def compute_score(y_pred, y):
    print("y: {}".format(y))
    print("y_pred: {}\n".format(y_pred))
    print("Accuracy: {}".format(metrics.accuracy_score(y_pred=y_pred, y_true=y)))
    print("Precision: {}".format(metrics.precision_score(y_pred=y_pred, y_true=y)))
    print("Recall: {}".format(metrics.recall_score(y_pred=y_pred, y_true=y)))
    print("F1: {}".format(metrics.f1_score(y_pred=y_pred, y_true=y)))

<h4>Scratch training</h4>

In [116]:
svm_scratch = ScratchSVMClassifier(num_iter=1000, lr=0.001, kernel='linear', threshold=1e-5, verbose=True, random_state=0)
svm_scratch.fit(X_train_std, y_train, X_test_std, y_test)
y_pred = svm_scratch.predict(X_test_std)

compute_score(y_pred, y_test)

KeyboardInterrupt: 

<h4>Sklearn training</h4>

In [ ]:
from sklearn.svm import SVC